\begin{align}
\large data_t &\large= data_{t-1} + {\Delta data} \quad \textrm{(number of data processed)} \tag{1} \\
\large uservalue_t &\large= data_t * \beta \quad \textrm{(amount of user value generated)} \tag{2}\\
\large usercost_t &\large= usercost_{t-1} + {\Delta cost} \quad \textrm{(amount of user cost incurred)} \tag{3}
\end{align}

If $\epsilon * provers$ > $\alpha$ the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta data} &\large= \alpha \quad \textrm{(data processed/ t)}
\end{align}

else:
\begin{align}
\large {\Delta data} &\large= \epsilon * provers \quad \textrm{(data processed/ t)}
\end{align}

If $\alpha$ > $\epsilon * provers$ the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta cost} &\large= \gamma * (\alpha - \epsilon * provers) \quad \textrm{(cost incurred/ t)}
\end{align}

else:
\begin{align}
\large {\Delta cost} &\large= 0 \quad \textrm{(cost incurred/ t)}
\end{align}

where:

$
\begin{align}
\alpha: \quad &\textrm{'data_inflow_rate'}\\
\epsilon: \quad &\textrm{'data_processing_rate'}\\
\beta: \quad &\textrm{'data_size_rate'}\\
\gamma: \quad &\textrm{'user_cost_rate'}\\
\end{align}
$

* Transactions (data) flow to the system at rate of $\alpha$ (alpha), transactions are processed at a rate proportional to the number of provers $\epsilon$ (epsilon).
* User value is generated proportional to the rate of data size $\beta$ (beta) of transactions processed.
* User cost is incurred proportional to the rate of user cost $\gamma$ (gamma) incurred due to unprocessed transactions.

In [40]:
import math
import pandas as pd
import plotly

pd.options.plotting.backend = "plotly"

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

In [89]:
initial_state = {
    'data': 0, # number of txns processed
    'uservalue': 0, # amount of user value generated
    'usercost': 0, # amount of user cost incurred 
}
system_params = {
    'data_inflow_rate': [20], # txn influx per minute
    'data_processing_rate': [5], # txns processing rate per prover
    'data_size_rate': [2], # data size of txns 
    'user_cost_rate': [4], # user cost is incurred due to unprocessed transactions
    'provers': [3], # number of provers 
}

In [90]:
def s_data(params, substep, state_history, previous_state, policy_input):
    data = previous_state['data'] + policy_input['delta_data'] 
    return 'data', max(data, 0)

def s_uservalue(params, substep, state_history, previous_state, policy_input):
    uservalue = params['data_size_rate'] * (previous_state['data'] + policy_input['delta_data']) 
    return 'uservalue', max(uservalue, 0)

def s_usercost(params, substep, state_history, previous_state, policy_input):
    usercost = previous_state['usercost'] + policy_input['delta_cost']
    return 'usercost', max(usercost, 0)

In [91]:
def p_data_inflow(params, substep, state_history, previous_state):
    if params['data_inflow_rate'] < (params['data_processing_rate'] * params['provers']):
        return {'delta_data': params['data_inflow_rate']}
    else: return {'delta_data': (params['data_processing_rate']* params['provers'])}

def p_user_cost(params, substep, state_history, previous_state):
    if params['data_inflow_rate'] > (params['data_processing_rate'] * params['provers']) :
        user_cost = params['user_cost_rate'] * (params['data_inflow_rate'] - (params['data_processing_rate'] * params['provers']))
        return {'delta_cost': user_cost}
    else: return {'delta_cost': 0}

In [92]:
partial_state_update_blocks = [
    {
        'policies': {
            'data_inflow': p_data_inflow,
            'user_cost': p_user_cost,
        },
        'variables': {
            'data': s_data,
            'uservalue': s_uservalue,
            'usercost': s_usercost
        }
    }
]

In [93]:
sim_config = config_sim({
    "N": 1,
    "T": range(10),
    "M": system_params
})

In [94]:
from cadCAD import configs
del configs[:]

experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=experiment.configs)

(raw_result, tensor_field, sessions) = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 5, 1, 3)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 5, 1, 3)
Execution Method: local_simulations
Execution Mode: single_threaded
Total execution time: 0.01s


In [95]:
simulation_result = pd.DataFrame(raw_result)

In [96]:
df = simulation_result.copy()
df = df[df.simulation == 0]
df

,data,uservalue,usercost,simulation,subset,run,substep,timestep
0,0,0,0,0,0,1,0,0
1,15,30,20,0,0,1,1,1
2,30,60,40,0,0,1,1,2
3,45,90,60,0,0,1,1,3
4,60,120,80,0,0,1,1,4
5,75,150,100,0,0,1,1,5
6,90,180,120,0,0,1,1,6
7,105,210,140,0,0,1,1,7
8,120,240,160,0,0,1,1,8
9,135,270,180,0,0,1,1,9


In [97]:
import plotly.express as px

px.line(df, x='timestep', y=['data', 'uservalue', 'usercost'])